In [66]:
import os
os.chdir("/Users/jessechuang/workspace/llm_learning")

from transformers import LlamaTokenizer
from model import RepeatModel
IGNORE_PADDING = 2

In [67]:
tokenizer = LlamaTokenizer.from_pretrained("models/tokenizer/merged_tokenizer_hf")
vocab_size = len(tokenizer)
max_length = 15
model = RepeatModel(vocab_size, 256, 8, 6, 6, max_length)    

In [92]:
import torch
model_path = "models/tokenizer/repeat_tranformer/repeat_model.pt"
model.load_state_dict(torch.load(model_path))
model.eval()

query = "Hi I am fine, and you</s>"
print("Query: {}".format(query))
a = tokenizer.encode(query)
src_tokens = torch.tensor(tokenizer.encode(query)).unsqueeze(0)
# input the first word of src for decoder
tgt_tokens = src_tokens[:,0].unsqueeze(0)

# Evaluation pipeline
output_string = ""
for i in range(max_length):
    out = model(src_tokens, tgt_tokens)
    # Only care about final output
    predict = model.predictor(out[:, -1])
    # Use Greedy Search for output
    y = torch.argmax(predict, dim=1)
    tgt_tokens = torch.concat([tgt_tokens, y.unsqueeze(0)], dim=1)
    output_token = tgt_tokens[0].numpy()
    output_string = tokenizer.decode(output_token)
    if output_token[-1] == IGNORE_PADDING:
        break
print("Answer: {}".format(output_string))

Query: Hi I am fine, and you</s>
Answer: <s> Hi I am fine, and you</s>
